In [1]:
import scanpy as sc
import numpy as np
import pandas as pd

In [2]:
from liana.method.sp._Misty import MistyData
from liana.method.sp._misty_constructs import genericMistyData, lrMistyData

In [3]:
adata = sc.read_h5ad('liana/tests/data/synthetic.h5ad')

In [4]:
misty = genericMistyData(intra=adata, set_diag=True, bandwidth=10)

/home/dbdimitrov/anaconda3/envs/liana-py/lib/python3.10/site-packages/mudata/_core/mudata.py:457: UserWarning: Cannot join columns with the same name because var_names are intersecting.


In [5]:
misty

MuData object with n_obs × n_vars = 4205 × 33
  obs:	'cell_type'
  3 modalities
    intra:	4205 x 11
      obs:	'cell_type'
      obsm:	'spatial'
    juxta:	4205 x 11
      obsm:	'spatial'
      obsp:	'spatial_connectivities'
    para:	4205 x 11
      obsm:	'spatial'
      obsp:	'spatial_connectivities'

In [6]:
misty(n_estimators=20, bypass_intra=True)

In [7]:
misty.uns['target_metrics']

,target,intra_group,env_group,intra.R2,multi.R2,gain.R2,juxta,para
0,ECM,None,None,0,0.233411,0.233411,0.773716,0.226284
1,ligA,None,None,0,0.504118,0.504118,0.832725,0.167275
2,ligB,None,None,0,0.443600,0.443600,0.813566,0.186434
3,ligC,None,None,0,0.508662,0.508662,0.694461,0.305539
4,ligD,None,None,0,0.511784,0.511784,0.834321,0.165679
5,protE,None,None,0,0.035423,0.035423,1.000000,0.000000
6,protF,None,None,0,0.131714,0.131714,0.865904,0.134096
7,prodA,None,None,0,0.052505,0.052505,1.000000,0.000000
8,prodB,None,None,0,0.077489,0.077489,1.000000,0.000000
9,prodC,None,None,0,0.069306,0.069306,1.000000,0.000000


In [8]:
misty.uns['importances']

,target,predictor,intra_group,extra_group,view,value
0,ECM,ligA,None,None,juxta,0.060664
1,ECM,ligB,None,None,juxta,0.076288
2,ECM,ligC,None,None,juxta,0.108785
3,ECM,ligD,None,None,juxta,0.067037
4,ECM,protE,None,None,juxta,0.233193
...,...,...,...,...,...,...
215,prodD,protE,None,None,para,0.104319
216,prodD,protF,None,None,para,0.097964
217,prodD,prodA,None,None,para,0.109447
218,prodD,prodB,None,None,para,0.090184


lrMistyData

In [9]:
from liana.testing._sample_anndata import generate_toy_spatial

In [10]:
adata = generate_toy_spatial()

In [11]:
adata.raw.X.tocsr()

<700x765 sparse matrix of type '<class 'numpy.float32'>'
	with 174400 stored elements in Compressed Sparse Row format>

In [12]:
lr_misty = lrMistyData(adata=adata, set_diag=True, bandwidth=10, use_raw=True)

/home/dbdimitrov/anaconda3/envs/liana-py/lib/python3.10/site-packages/mudata/_core/mudata.py:457: UserWarning: Cannot join columns with the same name because var_names are intersecting.


In [13]:
lr_misty()

In [15]:
lr_misty.uns['target_metrics']

,target,intra_group,env_group,intra.R2,multi.R2,gain.R2,intra,extra
0,CD4,None,None,-0.019744,0.007874,0.027618,0.000000,1.000000
1,ITGB2,None,None,0.070591,0.145086,0.074495,0.373496,0.626504
2,ITGA4,None,None,-0.009538,-0.008336,0.001202,NaN,NaN
3,HLA-DPB1,None,None,0.199734,0.873283,0.673550,0.017986,0.982014
4,CD63,None,None,-0.011116,0.004296,0.015412,0.412557,0.587443
5,CD33,None,None,0.137006,0.226625,0.089620,0.208899,0.791101
6,S1PR4,None,None,0.049102,0.076696,0.027593,0.495926,0.504074
7,CD53,None,None,-0.004101,0.016509,0.020610,0.225196,0.774804


In [16]:
lr_misty.uns['importances']

,target,predictor,intra_group,extra_group,view,value
0,CD4,ITGB2,None,None,intra,0.212282
1,CD4,ITGA4,None,None,intra,0.065106
2,CD4,HLA-DPB1,None,None,intra,0.195895
3,CD4,CD63,None,None,intra,0.135588
4,CD4,CD33,None,None,intra,0.090188
...,...,...,...,...,...,...
427,CD53,IL16,None,None,extra,0.042468
428,CD53,TIMP1,None,None,extra,0.068188
429,CD53,GNAI2,None,None,extra,0.041734
430,CD53,LTB,None,None,extra,0.097473


In [ ]:
from liana.resource import select_resource
from liana.method._pipe_utils import prep_check_adata
from liana.method.sp._lr_basis import _add_complexes_to_var
from liana.method.sp._misty_constructs import _make_view
from liana.method.sp._spatial_pipe import spatial_neighbors

In [ ]:
def lrMistyData(adata, 
                resource_name='consensus', 
                resource=None,
                nz_threshold=0.1,
                use_raw = False,
                layer = None,
                spatial_key='spatial',
                bandwidth = 100,
                kernel = 'misty_rbf',
                set_diag = True,
                cutoff = 0.1,
                zoi = 0,
                verbose = False
                ):
    
    if resource is None:
        resource = select_resource(resource_name.lower())
        
    adata = prep_check_adata(adata=adata,
                        use_raw=use_raw,
                        layer=layer,
                        verbose=verbose,
                        groupby=None,
                        min_cells=None,
                        obsm = {spatial_key: adata.obsm[spatial_key]}
                        )
    
    adata = _add_complexes_to_var(adata,
                              np.union1d(resource['receptor'].astype(str),
                                         resource['ligand'].astype(str)
                                        )
                              )
    
    views = dict()
    intra = _make_view(adata=adata[:, resource['receptor'].unique()],
                   nz_threshold=nz_threshold, obs=adata.obs)
    views['intra'] = _add_complexes_to_var(intra, np.unique(resource['receptor']))
    
    connectivity = spatial_neighbors(adata=adata, spatial_key=spatial_key,
                                 bandwidth=bandwidth, kernel=kernel,
                                 set_diag=set_diag, cutoff=cutoff,
                                 zoi=zoi, inplace=False)

    extra = _make_view(adata=adata[:,resource['ligand'].unique()],
                       spatial_key=spatial_key, nz_threshold=nz_threshold,
                       connecitivity=connectivity,
                       obs=pd.DataFrame(index=adata.obs.index))
    views['extra'] = _add_complexes_to_var(extra, np.unique(resource['ligand']))
    
    return MistyData(data=views, obs=intra.obs, spatial_key=spatial_key)

In [ ]:
use_raw = False
layer = None
verbose = True
resource = None
resource_name = 'consensus'
nz_threshold = 0.1

In [ ]:
spatial_key = 'spatial'
bandwidth = 100
kernel = 'misty_rbf'
set_diag = True
cutoff = 0.1
zoi = 0

In [ ]:
if resource is None:
    resource = select_resource(resource_name.lower())

In [ ]:
views = dict()

In [ ]:
adata = prep_check_adata(adata=adata,
                         use_raw=use_raw,
                         layer=layer,
                         verbose=verbose,
                         groupby=None,
                         min_cells=None,
                         obsm = {spatial_key: adata.obsm[spatial_key]}
                         )

In [ ]:
adata = _add_complexes_to_var(adata,
                              np.union1d(resource['receptor'].astype(str),
                                         resource['ligand'].astype(str)
                                        )
                              )

In [ ]:
adata

In [ ]:
# filter_resource
resource = resource[(np.isin(resource.ligand, adata.var_names)) &
                    (np.isin(resource.receptor, adata.var_names))]

In [ ]:
intra = _make_view(adata=adata[:, resource['receptor'].unique()],
                   nz_threshold=nz_threshold, obs=adata.obs)

views['intra'] = _add_complexes_to_var(intra, np.unique(resource['receptor']))

In [ ]:
connectivity = spatial_neighbors(adata=adata, spatial_key=spatial_key,
                                 bandwidth=bandwidth, kernel=kernel,
                                 set_diag=set_diag, cutoff=cutoff,
                                 zoi=zoi, inplace=False)

extra = _make_view(adata=adata[:,resource['ligand'].unique()], spatial_key=spatial_key,
                   nz_threshold=nz_threshold, connecitivity=connectivity, obs=pd.DataFrame(index=adata.obs.index))
views['extra'] = _add_complexes_to_var(extra, np.unique(resource['ligand']))

In [ ]:
from mudata import MuData

In [ ]:
lr_misty = MistyData(data=views, obs=intra.obs, spatial_key=spatial_key)

In [ ]:
lr_misty(bypass_intra=False)

In [ ]:
lr_misty.uns['target_metrics'].sort_values('multi.R2')

In [ ]:
lr_misty.uns['importances'].sort_values('value', ascending=False).head(20)